In [2]:
import pandas as pd
import json
import requests
from config import api_key
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify, render_template

In [4]:
headers = {'Authorization': 'Bearer %s' % api_key}

url='https://api.yelp.com/v3/businesses/search'
 
params = {'term':'restaurants', 
          'categories':'restaurants', 
          #'limit':1000, 
          'location':'94401',
          'radius': 5000}

response = requests.get(url, params=params, headers=headers)
print('The status code is {}'.format(response.status_code))
#if status code is 200, proceed.
#default returns 20 results unless limit specified in params

The status code is 200


In [6]:
result = json.loads(response.text)

In [7]:
search_result = result["businesses"]
search_result[0]

{'id': 'U9eYT-e179gIUb49IGK77A',
 'alias': 'kostantis-greek-kitchen-san-mateo',
 'name': 'Kostantis Greek Kitchen',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/hBkk1lywzP20ie55CoJsgw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/kostantis-greek-kitchen-san-mateo?adjust_creative=hRnZAfAqa_noY2v_VJB2Bg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hRnZAfAqa_noY2v_VJB2Bg',
 'review_count': 17,
 'categories': [{'alias': 'greek', 'title': 'Greek'},
  {'alias': 'mediterranean', 'title': 'Mediterranean'},
  {'alias': 'kebab', 'title': 'Kebab'}],
 'rating': 4.0,
 'coordinates': {'latitude': 37.565661, 'longitude': -122.322872},
 'transactions': ['pickup', 'delivery'],
 'location': {'address1': '250 S B St',
  'address2': None,
  'address3': '',
  'city': 'San Mateo',
  'zip_code': '94401',
  'country': 'US',
  'state': 'CA',
  'display_address': ['250 S B St', 'San Mateo, CA 94401']},
 'phone': '+16503985112',
 'display_phone': '(650) 398-511

In [8]:
business_name = []
business_review_count = []
avg_rating = []
lat = []
long = []
address = []
phone = []

for result in search_result:
    business_name.append(result["name"])
    business_review_count.append(result["review_count"])
    avg_rating.append(result["rating"])
    lat.append(result["coordinates"]["latitude"])
    long.append(result["coordinates"]["longitude"])
    address.append(result["location"]["display_address"])
    phone.append(result["display_phone"])
    
print(business_name)
print(business_review_count)
print(avg_rating)
print(lat)
print(long)
print(address)
print(phone)

['Kostantis Greek Kitchen', '3 Pigs Bbq', 'Green Leaf Bistro', 'Avenida Restaurant', 'Bangkok Patio', 'Yakitori Kokko', 'El Palenque Taqueria', 'Taishoken', 'Saltyard', 'Urban Momo', 'Chef Sha - San Mateo', 'Limón Rotisserie', 'Lobster Chef', 'Oh My WheelCake', 'Ramen Dojo', 'New Thai Elephant', 'Noodles & Things - San Mateo', 'Fire Wings', 'Zen Noodle Bar', 'Shihlin Taiwan Street Snacks']
[17, 56, 13, 71, 836, 972, 519, 585, 183, 121, 95, 1161, 36, 2, 3902, 200, 199, 439, 420, 70]
[4.0, 4.0, 5.0, 4.5, 4.0, 4.0, 4.5, 4.0, 4.0, 4.0, 4.5, 4.0, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, 4.0]
[37.565661, 37.5677018, 37.56366, 37.565194, 37.56767, 37.568120075232, 37.57463, 37.56333, 37.5796239823611, 37.5655985, 37.5654827296621, 37.57956, 37.5450627313526, 37.567369, 37.5621415150841, 37.56598, 37.564233, 37.5660924084269, 37.5672708, 37.56436]
[-122.322872, -122.324466756, -122.32556, -122.3231235, -122.31972, -122.321434020996, -122.31705, -122.3241, -122.346515189589, -122.3227471, -122.323286

In [9]:
result_df = pd.DataFrame({ "Business_Name" : business_name,
                           "Review_Count" : business_review_count,
                           "Average_Rating" : avg_rating,
                           "Latitude" : lat,
                           "Longtitude" : long,
                           "Address" : address,
                           "Phone_Number" : phone })
result_df = result_df.astype({"Business_Name": str, "Review_Count": int, "Average_Rating": float,
                              "Latitude": float, "Longtitude": float, "Address":str, "Phone_Number": str})
result_df

,Business_Name,Review_Count,Average_Rating,Latitude,Longtitude,Address,Phone_Number
0,Kostantis Greek Kitchen,17,4.0,37.565661,-122.322872,"['250 S B St', 'San Mateo, CA 94401']",(650) 398-5112
1,3 Pigs Bbq,56,4.0,37.567702,-122.324467,"['11 S B St', 'San Mateo, CA 94401']",(650) 389-6389
2,Green Leaf Bistro,13,5.0,37.563660,-122.325560,"['35 E 3rd Ave', 'San Mateo, CA 94401']",
3,Avenida Restaurant,71,4.5,37.565194,-122.323123,"['201 E 3rd Ave', 'San Mateo, CA 94401']",(650) 781-3637
4,Bangkok Patio,836,4.0,37.567670,-122.319720,"['615 E 3rd Ave', 'Ste D', 'San Mateo, CA 94401']",(650) 548-5489
5,Yakitori Kokko,972,4.0,37.568120,-122.321434,"['509 2nd Ave', 'San Mateo, CA 94401']",(650) 401-7008
6,El Palenque Taqueria,519,4.5,37.574630,-122.317050,"['15 N Kingston St', 'San Mateo, CA 94401']",(650) 342-4040
7,Taishoken,585,4.0,37.563330,-122.324100,"['47 E 4th Ave', 'San Mateo, CA 94401']",(650) 445-7579
8,Saltyard,183,4.0,37.579624,-122.346515,"['322 Lorton Ave', 'Burlingame, CA 94010']",(650) 342-7355
9,Urban Momo,121,4.0,37.565599,-122.322747,"['254 S B St', 'San Mateo, CA 94401']",(650) 458-3053


In [10]:
engine = create_engine("sqlite:///restaurant_db.sqlite")
conn = engine.connect()
Base = declarative_base()

In [11]:
class Restaurant(Base):
    __tablename__ = "restaurant_info"
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    Business_Name = Column(String(50))
    Review_Count = Column(Integer)
    Average_Rating = Column(Float)
    Latitude = Column(Float)
    Longtitude = Column(Float)
    Address = Column(String(200))
    Phone_Number = Column(String(30))

In [12]:
Base.metadata.create_all(conn)

In [13]:
result_df.to_sql("restaurant_info", con=engine, if_exists='replace', index_label='id')

In [14]:
pd.read_sql_query('select * from restaurant_info', con=engine).head()

,id,Business_Name,Review_Count,Average_Rating,Latitude,Longtitude,Address,Phone_Number
0,0,Kostantis Greek Kitchen,17,4.0,37.565661,-122.322872,"['250 S B St', 'San Mateo, CA 94401']",(650) 398-5112
1,1,3 Pigs Bbq,56,4.0,37.567702,-122.324467,"['11 S B St', 'San Mateo, CA 94401']",(650) 389-6389
2,2,Green Leaf Bistro,13,5.0,37.563660,-122.325560,"['35 E 3rd Ave', 'San Mateo, CA 94401']",
3,3,Avenida Restaurant,71,4.5,37.565194,-122.323123,"['201 E 3rd Ave', 'San Mateo, CA 94401']",(650) 781-3637
4,4,Bangkok Patio,836,4.0,37.567670,-122.319720,"['615 E 3rd Ave', 'Ste D', 'San Mateo, CA 94401']",(650) 548-5489
